### Word Group In Youtube

In [35]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [36]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [37]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 200  # 200  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select 
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
sentence_word_count_number = 6   # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

## prefix suffix file
#prefix_suffix = False  # True, False  # True for adding prefix suffix word
#native_word = True # True for adding native word

## adding output file extention
#if prefix_suffix & native_word:
#    file_ext = "5"
#elif (not prefix_suffix) & native_word:
#    file_ext = "6"
#else:
#    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

#print(f"{file_ext}")

In [38]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [39]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [40]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [41]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [42]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [43]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [44]:
## Option
#if prefix_suffix:
#    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
#    df_word = df_word.loc[:,["word","frequency"]]
#    df_word = pd.concat([df_word,df_word_select], axis=0)
#    df_word.drop_duplicates(inplace=True)    
#    df_word.sort_values(by="frequency", ascending=False, inplace=True)
#    df_word.reset_index(drop=True, inplace=True)
#else:
#    df_word = df_word_select
#
#if native_word:
#    df_word
#else:
#    df_word = df_word.head(0)
#
#df_word

In [45]:
df_word = df_word_select
df_word

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [46]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [47]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [48]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

200

#### Youtube Sentence Analysis

In [49]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:02.501,dit is de pangale v,BoaYsdPtJYA
1,00:00:04.501,00:00:05.434,unreal,BoaYsdPtJYA
2,00:00:08.518,00:00:13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,00:00:13.066,00:00:14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,00:00:14.316,00:00:15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...
252485,00:01:37.135,00:01:47.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,00:01:47.040,00:01:49.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,00:01:49.340,00:01:53.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,00:01:53.600,00:01:57.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [50]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [51]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA
1,1,4.501,5.434,unreal,BoaYsdPtJYA
2,2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA
3,3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA
4,4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA
...,...,...,...,...,...
252485,252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E
252486,252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E
252487,252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E
252488,252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E


In [52]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [53]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA,60.1,"v, pangale","dit, is, de"
1,1,4.501,5.434,unreal,BoaYsdPtJYA,0.1,unreal,
2,2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA,70.1,"supersport, eerste, rijd","in, is, een, keer, de, ik, dit"
3,3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA,85.8,hele,"in, mijn, leven, ik, dit, heb"
4,4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA,75.1,eerder,"gedaan, nooit, nog"
...,...,...,...,...,...,...,...,...
252485,252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E,37.6,"natte, vooral, soms, proberen, houden, kijken,...","te, met, de, het, zitten, we"
252486,252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E,66.8,"indrukwekkend, ziet, super","er, uit, en, was, dit, al"
252487,252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E,73.4,"circo, sinjor, vergeet, voorproefje","dus, zeker, naar, wel, is, een, nog, kom, geen..."
252488,252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E,60.1,"plaatsen, want, beperkt, tijdsblok, kiezen","zeker, ook, op, te, komen, tijd, en, de, zijn"


In [54]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA,60.1,"v, pangale","dit, is, de"
1,1,4.501,5.434,unreal,BoaYsdPtJYA,0.1,unreal,NaN
2,2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA,70.1,"supersport, eerste, rijd","in, is, een, keer, de, ik, dit"
3,3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA,85.8,hele,"in, mijn, leven, ik, dit, heb"
4,4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA,75.1,eerder,"gedaan, nooit, nog"
...,...,...,...,...,...,...,...,...
252485,252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E,37.6,"natte, vooral, soms, proberen, houden, kijken,...","te, met, de, het, zitten, we"
252486,252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E,66.8,"indrukwekkend, ziet, super","er, uit, en, was, dit, al"
252487,252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E,73.4,"circo, sinjor, vergeet, voorproefje","dus, zeker, naar, wel, is, een, nog, kom, geen..."
252488,252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E,60.1,"plaatsen, want, beperkt, tijdsblok, kiezen","zeker, ook, op, te, komen, tijd, en, de, zijn"


In [55]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,2.501,dit is de pangale v,BoaYsdPtJYA,60.1,"v, pangale","dit, is, de"
1,2,8.518,13.066,dit is de eerste keer ik rijd in een supersport,BoaYsdPtJYA,70.1,"supersport, eerste, rijd","in, is, een, keer, de, ik, dit"
2,3,13.066,14.316,ik heb dit in mijn hele leven,BoaYsdPtJYA,85.8,hele,"in, mijn, leven, ik, dit, heb"
3,4,14.316,15.889,nog nooit eerder gedaan,BoaYsdPtJYA,75.1,eerder,"gedaan, nooit, nog"
4,5,30.890,32.440,goedemorgen internet het is,BoaYsdPtJYA,50.1,"goedemorgen, internet","het, is"
...,...,...,...,...,...,...,...,...
235743,252485,97.135,107.040,soms zitten de volwassenen met natte ogen te k...,MZUDbceIw4E,37.6,"natte, vooral, soms, proberen, houden, kijken,...","te, met, de, het, zitten, we"
235744,252486,107.040,109.340,dit ziet er al was super indrukwekkend uit en,MZUDbceIw4E,66.8,"indrukwekkend, ziet, super","er, uit, en, was, dit, al"
235745,252487,109.340,113.600,het is nog maar een voorproefje dus kom zeker ...,MZUDbceIw4E,73.4,"circo, sinjor, vergeet, voorproefje","dus, zeker, naar, wel, is, een, nog, kom, geen..."
235746,252488,113.600,117.914,tijdsblok te kiezen en zeker ook op tijd te ko...,MZUDbceIw4E,60.1,"plaatsen, want, beperkt, tijdsblok, kiezen","zeker, ook, op, te, komen, tijd, en, de, zijn"


In [56]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
19,20,108.742,114.123,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,68.5,"grote, merk, liefde","dus, je, een, weet, dacht, voor, wat, en, het,...",en dus dacht ik dat je weet wat ik ga
20,22,116.302,119.175,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,69.3,"vragen, fiets","mijn, waar, moeten, zou, en, ben, ik, ga, ze",waar ik ben ik zou mijn
57,63,266.455,270.032,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,77.9,"vermogen, gereden","is, die, dan, ik, meer, al, heb",die al meer is dan ik heb
94,110,557.698,558.960,hoe dan ook zo in de,BoaYsdPtJYA,100.1,NaN,"in, ook, de, dan, hoe, zo",hoe dan ook zo in de
114,133,731.883,733.264,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA,72.8,"modi, drie, motor","in, nu, is, deze, de, en, het, heeft",en het is nu in de
...,...,...,...,...,...,...,...,...,...
235662,252368,843.770,850.460,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw,71.5,"harmonie, consequenties, totaliteit","in, die, dit, met, wat, even, de, we, zijn, over",zijn dit even wat over de
235664,252370,855.710,859.714,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw,63.7,"daarbij, vraag, antwoorden","nu, gaan, naar, de, en, we, over",nu gaan we over naar de
235734,252476,40.000,41.920,en wat is er allemaal te beleven dan,MZUDbceIw4E,87.6,beleven,"is, er, te, wat, en, allemaal, dan",en wat is er allemaal te
235740,252482,70.130,77.590,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,76.3,"vertellen, ingrid, repeteren","kan, je, dag, eens, wat, en, wie, jullie, hier...",wie zijn jullie en wat doen jullie


In [57]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,en dus dacht ik dat je weet wat ik ga,111.534671,114.123000,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA
1,waar ik ben ik zou mijn,116.302000,117.555673,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA
2,die al meer is dan ik heb,267.136333,269.435833,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA
3,hoe dan ook zo in de,557.698000,558.960000,hoe dan ook zo in de,BoaYsdPtJYA
4,en het is nu in de,732.680911,733.264000,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA
...,...,...,...,...,...
42270,zijn dit even wat over de,846.376494,848.722338,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw
42271,nu gaan we over naar de,855.710000,857.489556,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw
42272,en wat is er allemaal te,40.000000,41.333333,en wat is er allemaal te beleven dan,MZUDbceIw4E
42273,wie zijn jullie en wat doen jullie,75.194587,77.590000,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E


In [58]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,en dus dacht ik dat je weet wat ik ga,111.234671,114.423000,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA
1,waar ik ben ik zou mijn,116.002000,117.855673,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA
2,die al meer is dan ik heb,266.836333,269.735833,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA
3,hoe dan ook zo in de,557.398000,559.260000,hoe dan ook zo in de,BoaYsdPtJYA
4,en het is nu in de,732.380911,733.564000,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA
...,...,...,...,...,...
42270,zijn dit even wat over de,846.076494,849.022338,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw
42271,nu gaan we over naar de,855.410000,857.789556,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw
42272,en wat is er allemaal te,39.700000,41.633333,en wat is er allemaal te beleven dan,MZUDbceIw4E
42273,wie zijn jullie en wat doen jullie,74.894587,77.890000,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E


In [59]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA
4,en het is nu in de,732,734,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA
...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E


In [60]:
df_word_group_time_loc_in_sent_result.sort_values("search_string", key=lambda x:x.str.len(), inplace=True, ascending=True)  # ascending=False
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA
4,en het is nu in de,732,734,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA
...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E


In [61]:
df_word_group_time_loc_in_sent_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_sent_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_sent_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,en dus dacht ik dat je weet wat ik ga,111,114,dus ik heb een grote liefde voor het merk en d...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
1,waar ik ben ik zou mijn,116,118,waar ik ben ik zou mijn fiets moeten en ik ga ...,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
2,die al meer is dan ik heb,267,270,vermogen die al meer is dan ik heb gereden,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
3,hoe dan ook zo in de,557,559,hoe dan ook zo in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
4,en het is nu in de,732,734,deze motor heeft drie modi en het is nu in de,BoaYsdPtJYA,https://www.youtube.com/watch?v=BoaYsdPtJYA&t=...
...,...,...,...,...,...,...
42270,zijn dit even wat over de,846,849,in harmonie met die totaliteit zijn dit even w...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
42271,nu gaan we over naar de,855,858,nu gaan we over naar de vraag en de antwoorden...,z8hxInj0hPw,https://www.youtube.com/watch?v=z8hxInj0hPw&t=...
42272,en wat is er allemaal te,40,42,en wat is er allemaal te beleven dan,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=40s
42273,wie zijn jullie en wat doen jullie,75,78,dag ingrid ik heb jullie al zien repeteren hie...,MZUDbceIw4E,https://www.youtube.com/watch?v=MZUDbceIw4E&t=75s


In [62]:
df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)

In [63]:
df_word_group_time_loc_in_sent_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word.csv", index=False)

In [64]:
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word.xlsx", index=False)

#### Youtube Videoid Text Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=word_group_adjust_select, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
#df_word_group_time_loc_in_text_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word.xlsx", index=False)

#### Copy Move And Delete

In [65]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*")
output_file

['Dutch_Word_Group_Max_In_Youtube_Sentence_200_Word.csv']

In [66]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [67]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Adjust Result Analysis Multiprocess

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = True  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to all word group length

# # shared word frequency
# shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list3 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list3.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list3), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_text_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
        
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [ ]:
df_adjust_text_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_text_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_text_ratio.reset_index(drop=True, inplace=True)
df_adjust_text_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_All_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass